In [ ]:
# Importing necessary modules
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
# Setting up the URLs (four pages of one table) to scrape from  
urls = ['https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2020&year_max=2020&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=g&c1comp=gt&c1val=25&c2stat=mp_per_g&c2comp=gt&c2val=12&c3stat=def_rtg&c3comp=gt&c3val=&c4stat=usg_pct&c4comp=gt&c4val=&c5stat=def_rtg&c5comp=gt&c6mult=&c6stat=&order_by=dws&order_by_asc=&offset=0', 
        'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2020&year_max=2020&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=g&c1comp=gt&c1val=25&c2stat=mp_per_g&c2comp=gt&c2val=12&c3stat=def_rtg&c3comp=gt&c3val=&c4stat=usg_pct&c4comp=gt&c4val=&c5stat=def_rtg&c5comp=gt&c6mult=&c6stat=&order_by=dws&order_by_asc=&offset=100', 
        'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2020&year_max=2020&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=g&c1comp=gt&c1val=25&c2stat=mp_per_g&c2comp=gt&c2val=12&c3stat=def_rtg&c3comp=gt&c3val=&c4stat=usg_pct&c4comp=gt&c4val=&c5stat=def_rtg&c5comp=gt&c6mult=&c6stat=&order_by=dws&order_by_asc=&offset=200',
        'https://www.basketball-reference.com/play-index/psl_finder.cgi?request=1&match=single&type=per_game&per_minute_base=36&per_poss_base=100&lg_id=NBA&is_playoffs=N&year_min=2020&year_max=2020&franch_id=&season_start=1&season_end=-1&age_min=0&age_max=99&shoot_hand=&height_min=0&height_max=99&birth_country_is=Y&birth_country=&birth_state=&college_id=&draft_year=&is_active=&debut_yr_nba_start=&debut_yr_nba_end=&is_hof=&is_as=&as_comp=gt&as_val=0&award=&pos_is_g=Y&pos_is_gf=Y&pos_is_f=Y&pos_is_fg=Y&pos_is_fc=Y&pos_is_c=Y&pos_is_cf=Y&qual=&c1stat=g&c1comp=gt&c1val=25&c2stat=mp_per_g&c2comp=gt&c2val=12&c3stat=def_rtg&c3comp=gt&c3val=&c4stat=usg_pct&c4comp=gt&c4val=&c5stat=def_rtg&c5comp=gt&c6mult=&c6stat=&order_by=dws&order_by_asc=&offset=300']

# creating empty lists outside of loop to use inside the loop, and store data in.
all_stats_byrow = []
player_names_list = []
temp = []
    
for url in urls:
    r_bbref_3d = rq.get(url)
    stats_bbref_soup = BeautifulSoup(r_bbref_3d.text, 'lxml')
    
    # Extracting table headers
    look_for_headers = stats_bbref_soup.table.thead.find_all('tr')
    look_for_headers[:1]
    headers_list = look_for_headers[1].text
    headers_list = headers_list.split('\n')
    headers_list = headers_list[2:-1]
        
    # Extracting all data from the table, row by row
    every_row = stats_bbref_soup.table.tbody.find_all('tr')  
    for row in every_row:
        list_of_stats = row.find_all('td')
        for td in list_of_stats:
            stat = td.text
            temp.append(stat)
        all_stats_byrow.append(temp)
        temp = []
        
# Cleaning the data. Fixing NAs, replacing missing data, and changing data types for certain columns.
allstats_df = pd.DataFrame(data=all_stats_byrow, columns=headers_list)
allstats_df = allstats_df.dropna().set_index('Player')
allstats_df.replace('','0', inplace=True)
allstats_df['G'] = allstats_df['G'].astype(int)
list_of_columns_ToBeFloats = allstats_df.columns[5:]
for col in list_of_columns_ToBeFloats:
    allstats_df[col] = allstats_df[col].astype(float)

allstats_df

In [ ]:
# Exporting the data to a csv file
allstats_df.to_csv('bbref_3&d&usg.csv')